In [1]:
import sys
sys.path.append('../../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np
import time

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
import mthree
from qiskit.test.mock import FakeManhattan
from libs_qrem import sgs_algorithm_py

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_14753/482182139.py:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import TensoredMeasFitter


In [3]:
max_iter = 10
min_size = 1
max_size = 30

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitter_list_list.pkl", "rb") as f:
    meas_fitter_list_list = pickle.load(f)

In [5]:
mthree_lu_mitigator_list_list = []
for num_qubits in range(min_size, max_size + 1):
    mthree_lu_mitigator_list = []
    for i in range(0, max_iter):
        mit = mthree.M3Mitigation(FakeManhattan())
        mit.single_qubit_cals = meas_fitter_list_list[num_qubits - 1][i].cal_matrices
        mthree_lu_mitigator_list.append(mit)
        if i % 10 == 0:
            print("iter", i, "finished")
    mthree_lu_mitigator_list_list.append(mthree_lu_mitigator_list)
    print("length of mthree_lu_mitigator_list: ", len(mthree_lu_mitigator_list))
    print("size", num_qubits, "finished")
    print()

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 1 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 2 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 3 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 4 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 5 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 6 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 7 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 8 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 9 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 10 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 11 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 12 finished

iter 0 finished
length of mthree_lu_mitigator_list:  10
size 13 finished

iter 0 finished
length of mthree_lu_mitigator_l

In [7]:
mthree_lu_info_list_list = []
mthree_lu_hist_list_list = []
for i in range(min_size - 1, max_size):
    mthree_lu_info_list = []
    mthree_lu_hist_list = []
    for j in range(0, max_iter):
        meas_layout = [k for k in range(i + 1)]
        mit_hist, info = mthree_lu_mitigator_list_list[i][j].apply_correction(raw_hist_list_list[i][j],
                                                                      meas_layout, 
                                                                      distance=None,
                                                                      method='direct',
                                                                      max_iter=25, 
                                                                      tol=1e-5,
                                                                      return_mitigation_overhead=True,
                                                                      details=True)
        mthree_lu_hist_list.append( sgs_algorithm_py(mit_hist) )
        mthree_lu_info_list.append( info )
        print("iter", j, "finished")
    mthree_lu_hist_list_list.append( mthree_lu_hist_list )
    mthree_lu_info_list_list.append( mthree_lu_info_list )
    print("size", i + 1, "finished")
    print()

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 1 finished

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 2 finished

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 3 finished

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 4 finished

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
size 5 finished

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
ite

In [8]:
def rescale_hist(hist, shots):
    new_hist = dict()
    for k, v in hist.items():
        new_hist[k] = v * shots
    return new_hist

In [9]:
for i in range(min_size - 1, max_size):
    for j in range(0, max_iter):
        mthree_lu_hist_list_list[i][j] = rescale_hist(mthree_lu_hist_list_list[i][j], 1 << 13)

In [10]:
with open("pkls/mthree_lu_info_list_list.pkl", "wb") as f:
    pickle.dump(mthree_lu_info_list_list, f)
with open("pkls/mthree_lu_hist_list_list.pkl", "wb") as f:
    pickle.dump(mthree_lu_hist_list_list, f)